# Student Loan Risk with Deep Learning

In [68]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [69]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [70]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [71]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [72]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
print(f"target shape{y.shape}")

target shape(1599,)


In [73]:
from re import X
# Define features set X by selecting all columns but credit_ranking
x = loans_df.drop(columns= ["credit_ranking"])
print(x)

# Review the features DataFrame
x.shape

      payment_history  location_parameter  stem_degree_score  gpa_ranking  \
0                 7.4               0.700               0.00          1.9   
1                 7.8               0.880               0.00          2.6   
2                 7.8               0.760               0.04          2.3   
3                11.2               0.280               0.56          1.9   
4                 7.4               0.700               0.00          1.9   
...               ...                 ...                ...          ...   
1594              6.2               0.600               0.08          2.0   
1595              5.9               0.550               0.10          2.2   
1596              6.3               0.510               0.13          2.3   
1597              5.9               0.645               0.12          2.0   
1598              6.0               0.310               0.47          3.6   

      alumni_success  study_major_code  time_to_completion  \
0            

(1599, 11)

### Step 3: Split the features and target sets into training and testing datasets.


In [74]:
# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42, stratify=y)
# Assign the function a random_state equal to 1
print(f"Training Features Shape: {x_train.shape}")
print(f"Training Features Shape: {x_test.shape}")
print(f"Training Features Shape: {y_train.shape}")
print(f"Training Features Shape: {y_test.shape}")

Training Features Shape: (1279, 11)
Training Features Shape: (320, 11)
Training Features Shape: (1279,)
Training Features Shape: (320,)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [75]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
x_train_scaled = scaler.fit_transform(x_train)

# Fit the scaler to the features training dataset
x_test_scaled = scaler.transform(x_test)

x_train_scaled = pd.DataFrame(x_train_scaled,columns=x.columns)
x_test_scaled = pd.DataFrame(x_test_scaled,columns=x.columns)

x_train_scaled.head()



,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,-0.590966,-0.088985,-0.479265,-0.318464,-0.362708,-0.376379,-0.555343,0.485602,1.350662,0.432631,-0.874168
1,-0.932515,0.078962,-0.737340,-0.390869,-0.232593,-0.082760,-0.314237,0.288102,1.935417,-0.111633,-0.594279
2,0.433678,-0.144968,0.140115,-0.463274,-0.471137,-0.767871,-0.886864,0.516786,0.051206,0.311684,0.058793
3,-0.932515,1.310575,-1.305106,-0.535679,-0.189221,-0.963618,-1.037555,-0.387555,1.545580,-0.172107,-0.454335
4,-0.647891,0.806733,-1.408336,-0.246059,-0.406080,-0.572125,-0.675896,-0.595449,0.700934,0.372157,0.618570


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [76]:
# Define the the number of inputs (features) to the model
input_features=x_train_scaled.shape[1]



In [77]:
# Define the number of hidden nodes for the first hidden layer
hidden_notes_layer1= 16


# Define the number of hidden nodes for the second hidden layer
hidden_notes_layer2= 8

# Define the number of neurons in the output layer
output_neurons=1

In [78]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(hidden_notes_layer1, activation = 'relu', input_shape=(input_features,)))

# Add the second hidden layer
model.add(Dense(hidden_notes_layer2, activation = 'relu'))


# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(output_neurons, activation = 'sigmoid'))

/opt/anaconda3/envs/dev/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [79]:
# Display the Sequential model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 16)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 337 (1.32 KB)

 Trainable params: 337 (1.32 KB)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [80]:
# Compile the Sequential model
model.compile(loss= "binary_crossentropy", optimizer = 'adam', metrics = ["accuracy"])

In [81]:
# Fit the model using 50 epochs and the training data
history= model.fit(x_train_scaled, y_train, epochs=50,batch_size=32, validation_data=(x_test_scaled,y_test))

Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6240 - loss: 0.6718 - val_accuracy: 0.6969 - val_loss: 0.5942
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6592 - loss: 0.6083 - val_accuracy: 0.7031 - val_loss: 0.5600
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7073 - loss: 0.5695 - val_accuracy: 0.7031 - val_loss: 0.5400
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7288 - loss: 0.5438 - val_accuracy: 0.7063 - val_loss: 0.5265
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7373 - loss: 0.5285 - val_accuracy: 0.7312 - val_loss: 0.5168
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7491 - loss: 0.5072 - val_accuracy: 0.7250 - val_loss: 0.5111
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7305 - loss: 0.5274 - val_accuracy: 0.7219 - val_loss: 0.5060
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7618 - loss: 0.4890 - val_accuracy: 0.7250 - val_loss:

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [82]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(x_test_scaled,y_test)
# Display the model loss and accuracy results

print(f"Model Loss: {loss:.4f}")
print(f"Model accuracy: {accuracy:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7631 - loss: 0.5029
Model Loss: 0.5009
Model accuracy: 0.7656


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [83]:
model_file_path = "student_loans.keras"  # You can change this path if needed

# Export your model to a keras file
model.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [84]:
# Set the model's file path
model_file_path= "student_loans.keras"
model.save(model_file_path)

# Load the model to a new object


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [85]:
# Make predictions with the test data
model_file_path = "student_loans.keras"
loaded_model = load_model(model_file_path)

# Display a sample of the predictions
loaded_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 16)             │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,013 (3.96 KB)

 Trainable params: 337 (1.32 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 676 (2.64 KB)

In [86]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions = loaded_model.predict(x_test_scaled)
predicted_classes = (predictions>0.5).astype(int)
sample_predictions = predicted_classes[:10]
print(sample_predictions.flatten())


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
[0 0 0 0 1 1 0 0 0 1]


### Step 4: Display a classification report with the y test data and predictions

In [87]:
# Print the classification report with the y test data and predictions
predictions_df = pd.DataFrame({"actual": y_test.values, "predicted" : predicted_classes.flatten() })
print(predictions_df.head())


report= classification_report(y_test,predicted_classes)
print(report)

   actual  predicted
0       1          0
1       0          0
2       0          0
3       0          0
4       1          1
              precision    recall  f1-score   support

           0       0.71      0.83      0.77       149
           1       0.83      0.71      0.76       171

    accuracy                           0.77       320
   macro avg       0.77      0.77      0.77       320
weighted avg       0.77      0.77      0.77       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

## **1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

